In [263]:
%pip install numpy
%pip install pandas
%pip install matplotlib

81062.51s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.
81069.84s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.
81076.76s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


In [174]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
import matplotlib.pyplot as plt
import re
import json


At some point while writing this i accidentally deleted the file and had to rewrite parts of it. 
You can do ur best guess on when this happened by looking at the amount of comments in the code.

In [260]:
# Let movies be a seperate array for easier access later.
movies = ["The Dark Knight", "Interstellar", "Star-wars", "Løvernes Konge", "De urørlige", "The prestige"]

# fact_table = [
#     ["",          *movies], # Unwrap the movies array with spread operator.
#     ["Anton",     4, 5, 4, 2, 3, 4],
#     ["Benjamin",  3, 2, 2, 4, 5, 3],
#     ["Mads",      4, 2, 4, 3, 3, 5],
#     ["Joachim",   4, 4, 5, 3, 3, 4],
#     ["Rasmus",    2, 1, 2, 5, 4 ,3],
#     ["Peter",     5, 5, 3, 1, 2, 4],
# ]

fact_table = [
    ["",          *movies], # Unwrap the movies array with spread operator.
    ["Anton",     4, 5, 4, 2, 3, 4],
    ["Benjamin",  3, 2, 2, 4, 5, 3],
    ["Mads",      4, 2, 4, 3, 3, 5],
    ["Joachim",   4, 4, 5, 3, 3, 4],
    ["Rasmus",    2, 1, 2, 5, 4 ,3],
    ["Peter",     5, 5, 3, 1, 2, 4],
]


# Makes a mapping from index space of fact_matrix to human readable labels and inverse.
# (The * operator here is called the spread operator and the syntax for creating dicts and lists here is called list comprehension)
users = [ user for user in (fact_table[i][0] for i in range(len(fact_table)))]
users = { users[i+1]: i for i in range(len(users[1:])) } # We downshift index mappings by 1 because indexes start from 0 not 1.
inv_users = { v: k for (k,v) in users.items() }

# Set up the fact table to a pandas dataframe.
fact_matrix = pd.DataFrame(fact_table)
fact_matrix.header = fact_matrix.iloc[0:1] # sets the header to the first row 
fact_matrix.columns = fact_matrix.iloc[0] # Give the correct headers
fact_matrix = fact_matrix[1:] # Remove the header from row space
fact_matrix[movies] # Perfect matrix presentation.

def prettier_dataframe_to_latex(df: pd.DataFrame, noHead = False, noIndex = False) -> str:
    result = df.style

    if noIndex and noHead:
        # Remove header row and index column
        result = df.style.format("{:.1f}").hide().hide(axis=1)
    else: 
        if noIndex:
            # Remove index column
            result = df.style.format("{:.1f}").hide()
        if noHead:
            # Remove header
            result = df.style.format().hide(axis=1)

    result = result.format(escape="latex").to_latex().replace("tabular","pmatrix")
    return  re.sub(r'\{l+\}', "", result) # Regex away the leftover column space rendering hints from tabular output element

# You can uncomment each line to see what they look like or simply browse the next section.
# print(prettier_dataframe_to_latex(fact_matrix))
# print(prettier_dataframe_to_latex(fact_matrix[movies]))
# print(prettier_dataframe_to_latex(fact_matrix, noHead=True, noIndex=True))
# print(prettier_dataframe_to_latex(fact_matrix[movies], noHead=True, noIndex=True))

In [142]:
def pearson_similarity(vec_a: np.array, vec_b: np.array) -> float:
    # Returns the pearson similarity between two vectors.
    vec_a = vec_a - vec_a.mean() # Compute mean norm of vec_a
    vec_b = vec_b - vec_b.mean() # Compute mean norm of vec_b
    return np.dot(vec_a, vec_b) / (LA.norm(vec_a) * LA.norm(vec_b))

def manthattan_similarity(vec_a: np.array, vec_b: np.array) -> float:
    # Returns the manhattan similarity between two vectors
  return  1/ (1+ np.sum(np.abs(vec_a - vec_b)))

def eucledian_similarity(vec_a, vec_b):
    # Returns the eucledian similarity between two vectors
    return 1 / (1 + np.sqrt(np.sum(np.power(vec_a - vec_b,2))))

In [143]:
def average_prediction(fact_matrix: np.matrix, user_movie_subset: np.matrix) -> np.matrix:
    prediciton_matrix = np.empty(fact_matrix[movies].shape)
    for i in range(fact_matrix[movies].shape[0]):
        rows = []
        for k in user_movie_subset[str(i)+"-idx"]:
            rows.append(
                fact_matrix[movies].iloc[k,:]
            )
        # Compute average of rows by column.
        averages_of_movies_from_nearest_neighbours = pd.DataFrame(rows).mean(axis=0)
        for j in range(len(averages_of_movies_from_nearest_neighbours)):
            prediciton_matrix[i,j] = averages_of_movies_from_nearest_neighbours[j]

    return prediciton_matrix

def weighted_average_prediction(fact_matrix: np.matrix, user_movie_subset: np.matrix) -> np.matrix:
    # Here i want to elementwise multiply the weight with the output of the average_prediction function.
    prediciton_matrix = np.empty(fact_matrix[movies].shape)
    for i in range(fact_matrix[movies].shape[0]):
        similarity_sum = user_movie_subset[str(i)].sum()
        weight_index_dict = {}
        for k in range(len(user_movie_subset[str(i)])):
            weight_index_dict[user_movie_subset[str(i)+"-idx"][k]] = user_movie_subset[str(i)][k] / similarity_sum
        # Compute weighted rating of movies from nearest neighbours.
        for key in weight_index_dict.keys():
            prediciton_matrix[i,:] += fact_matrix[movies].iloc[key,:] * weight_index_dict[key]
                        
    return prediciton_matrix

def weighted_average_corrected_prediction(fact_matrix: np.matrix, user_movie_subset: np.matrix) -> np.matrix:
    prediciton_matrix = np.empty(fact_matrix[movies].shape)
    for i in range(fact_matrix[movies].shape[0]):
        similarity_sum = user_movie_subset[str(i)].sum()
        weight_index_dict = {}
        for k in range(len(user_movie_subset[str(i)])):
            weight_index_dict[user_movie_subset[str(i)+"-idx"][k]] = user_movie_subset[str(i)][k] / similarity_sum
        # Compute weighted rating of movies from nearest neighbours.
        for key in weight_index_dict.keys():
            Ra_avg = np.mean(fact_matrix[movies].iloc[i,:]) 
            Rb_avg = np.mean(fact_matrix[movies].iloc[key,:]) 
            # print(Ra_avg)
            # print((fact_matrix[movies].iloc[key,:] - Rb_avg) * weight_index_dict[key])
            prediciton_matrix[i,:] += (Ra_avg / len(weight_index_dict)) + ((fact_matrix[movies].iloc[key,:] - Rb_avg) * weight_index_dict[key])
                        
    return prediciton_matrix

In [145]:
def similarity_matrix(input_matrix: np.matrix, similarity_function: callable, column_major = True) -> np.matrix:
    # Computes the similarity matrix for the input matrix.
    similarity_matrix = np.identity((input_matrix.shape[0])) # Make the matrix N x N with a diagonal of 1 as users are perfectly similar to themself

    for i in range(input_matrix.shape[0]):
        for j in range(input_matrix.shape[0]):
            if i==j:
                # Skips current iteration because the result is known beforehand to be 1.
                # Could also speed up the computation by a factor of 2 by recognising that these matrices are all symmetric around the diagonal because of the 
                # commutative property of the similarity measures used. In which case we could use the break keyword instead followed by a transpose statement in the output.
                continue
            # if column major it will apply two columns and find the similarity between those two
            if column_major: 
                similarity_matrix[i, j] = similarity_function(input_matrix[movies[i]], input_matrix[movies[j]])
            # If row major it will apply the same but to different rows
            else:
                similarity_matrix[i, j] = similarity_function(input_matrix[movies].iloc[i], input_matrix[movies].iloc[j])
    resulting_matrix = np.matrix(similarity_matrix)
    # This can be skipped if you use continue instead of break earlier. But what would the fun in that be?
    return resulting_matrix
    # return resulting_matrix + resulting_matrix.transpose() - np.identity(resulting_matrix.shape[0]) # Slightly faster implementation for the lols

# You will notice that this step is left out of the homework assignment but 
# programatically this works out sort of like glue which allows you to go 
# from one step to another without overly complicating the other functions 
# which i would really prefer to just do one thing onto itself, one function at a time.
def knn_tuple(similarity_matrix: np.matrix, k=1) -> np.array:
    # Maps every index with their k highest value indices
    temp_sorted_arrays = {}
    sm_df = pd.DataFrame(similarity_matrix)

    for i in range(similarity_matrix.shape[0]):
        # The syntax here is quite werid because i use a chain of python tricks
        # And type castings to access functions that id rather use over others
        # The result is the list of k-most nearest neighbours
        temp_sorted_array = sm_df.iloc[i].sort_values()[::-1].array[1:]
        temp_sorted_arrays[str(i)] = temp_sorted_array[0:k]
        element_wise_mapping = []
        for j in temp_sorted_arrays[str(i)]:
            # Technically here we are assuming that this function is invertible.
            # Failing this assumption we will ignore two neighbours of same distance
            # Which is not always true, it just so happens that it becomes true more often with 
            # more features (movies) or if the distance happens to be equal to two neighbours
            # whose data is also equal.
            # No such instances of equidistant neighbours occur with out dataset.
            element_wise_mapping.append(list(sm_df.iloc[i]).index(j))
        temp_sorted_arrays[str(i)+"-idx"] = element_wise_mapping

    # This returns two things. A list of the k highest values and their index on the row space of the similarity matrix
    # You can feed this index into inv_users for names or use iloc calls to fact_matrix to generate predictions.
    # Maps kind of like this: distances -> user <- users
    return temp_sorted_arrays

def prediction_matrix(fact_matrix: np.matrix, knn_tuple: dict, prediction_funciton: callable) -> np.matrix:
    # Computes the prediction matrix for the input matrix given by similarity matrix + some KNN regieme.#
    return prediction_funciton(fact_matrix, knn_tuple)

In [183]:
def mean_absolute_error(fact_matrix: np.matrix, prediction_matrix: np.matrix) -> float:
    error_matrix = fact_matrix[movies] - prediction_matrix
    return np.sum(np.sum(np.abs(error_matrix)) / np.count_nonzero(error_matrix))

def mean_absolute_error_squared(fact_matrix: np.matrix, prediction_matrix: np.matrix) -> float:
    error_matrix = fact_matrix[movies] - prediction_matrix
    return np.sum(np.sum(np.power(error_matrix,2)) / np.count_nonzero(error_matrix))

def absolute_mean_residual_squared(fact_matrix: np.matrix, prediction_matrix: np.matrix) -> float:
    error_matrix = (fact_matrix[movies] - prediction_matrix)**2
    return np.sum(np.sum(error_matrix) / np.count_nonzero(error_matrix))


In [199]:
gs_results = {}

error_functions = [mean_absolute_error, mean_absolute_error_squared, absolute_mean_residual_squared]
prediction_functions = [average_prediction, weighted_average_prediction, weighted_average_corrected_prediction]
similarity_functions = [pearson_similarity, manthattan_similarity, eucledian_similarity]
fn_name_from_index = lambda fn_list, k:  str(fn_list[k].__str__().split(" ")[1])
# print(fn_name_from_index(error_functions, 1))

def compute_predictions(error_function: callable, prediction_function: callable, similarity_function: callable, fact_matrix: np.matrix, k_nearest: int) -> float:
    # This function is very much a case of just call one method after another. Here you see why i write descriptive names for my variables and arguments.
    sim_mat = similarity_matrix(
        fact_matrix, 
        similarity_function=similarity_function,
        column_major=False
    )
    knn = knn_tuple(
        similarity_matrix=sim_mat,
        k=k_nearest
    )
    pred_mat = prediction_matrix(
        fact_matrix=fact_matrix,
        knn_tuple=knn,
        prediction_funciton=prediction_function
    )
    return error_function(fact_matrix=fact_matrix, prediction_matrix=pred_mat)

def grid_search(fact_matrix):
    global gs_results # call global here to exit block scope of grid_search()
    # 3^3*5 = 135 experiments
    for i in range(len(error_functions)):
        for j in range(len(prediction_functions)):
            for s in range(len(similarity_functions)):
                for k in range(1,5):
                    gs_results[
                        f"err={fn_name_from_index(error_functions, i)} pre={fn_name_from_index(prediction_functions,j)} sim={fn_name_from_index(similarity_functions,s)} k-near={k}"
                    ] = compute_predictions(
                        error_function=error_functions[i],
                        prediction_function=prediction_functions[j],
                        similarity_function=similarity_functions[s],
                        fact_matrix=fact_matrix,
                        k_nearest=k
                    )
    return gs_results

grid_search(fact_matrix)

# Save gs_results to file
    

pd.DataFrame(gs_results, index=[0]).sort_values(by=0, axis=1).to_csv("gs_results.csv")

In [192]:
test_sim = similarity_matrix(fact_matrix, similarity_function=pearson_similarity, column_major=False)
predmat = prediction_matrix(fact_matrix, knn_tuple(test_sim, k=3), prediction_funciton=average_prediction)
mean_absolute_error(fact_matrix,predmat)

0.9583333333333331

In [167]:
test_sim = similarity_matrix(fact_matrix, similarity_function=pearson_similarity, column_major=False)
knn = knn_tuple(test_sim, k=3)

weighted_average_prediction(
  
)

0.9157295698813058

In [321]:
# Just manually using the functions to compute the results for the report.

XAnton = [
    ["",          *movies], # Unwrap the movies array with spread operator.
    ["Anton",     4, 0, 4, 2, 3, 4],
    ["Benjamin",  3, 2, 2, 4, 5, 3],
    ["Mads",      4, 2, 4, 3, 3, 5],
    ["Joachim",   4, 4, 5, 3, 3, 4],
    ["Rasmus",    2, 1, 2, 5, 4 ,3],
    ["Peter",     5, 5, 3, 1, 2, 4],
]
XAnton = pd.DataFrame(fact_table)
XAnton.header = XAnton.iloc[0:1] # sets the header to the first row 
XAnton.columns = XAnton.iloc[0] # Give the correct headers
XAnton = XAnton[1:] # Remove the header from row space
XAnton[movies] # Perfect matrix presentation.

XBenjamin = [
    ["",          *movies], # Unwrap the movies array with spread operator.
    ["Anton",     4, 5, 4, 2, 3, 4],
    ["Benjamin",  3, 2, 0, 4, 5, 3],
    ["Mads",      4, 2, 4, 3, 3, 5],
    ["Joachim",   4, 4, 5, 3, 3, 4],
    ["Rasmus",    2, 1, 2, 5, 4 ,3],
    ["Peter",     5, 5, 3, 1, 2, 4],
]
XBenjamin = pd.DataFrame(fact_table)
XBenjamin.header = XBenjamin.iloc[0:1] # sets the header to the first row 
XBenjamin.columns = XBenjamin.iloc[0] # Give the correct headers
XBenjamin = XBenjamin[1:] # Remove the header from row space
XBenjamin[movies] # Perfect matrix presentation.

XMads = [
    ["",          *movies], # Unwrap the movies array with spread operator.
    ["Anton",     4, 5, 4, 2, 3, 4],
    ["Benjamin",  3, 2, 2, 4, 5, 3],
    ["Mads",      4, 2, 4, 3, 3, 0],
    ["Joachim",   4, 4, 5, 3, 3, 4],
    ["Rasmus",    2, 1, 2, 5, 4 ,3],
    ["Peter",     5, 5, 3, 1, 2, 4],
]
XMads = pd.DataFrame(fact_table)
XMads.header = XMads.iloc[0:1] # sets the header to the first row 
XMads.columns = XMads.iloc[0] # Give the correct headers
XMads = XMads[1:] # Remove the header from row space
XMads[movies] # Perfect matrix presentation.

XJoachim = [
    ["",          *movies], # Unwrap the movies array with spread operator.
    ["Anton",     4, 5, 4, 2, 3, 4],
    ["Benjamin",  3, 2, 2, 4, 5, 3],
    ["Mads",      4, 2, 4, 3, 3, 5],
    ["Joachim",   0, 4, 5, 3, 3, 4],
    ["Rasmus",    2, 1, 2, 5, 4 ,3],
    ["Peter",     5, 5, 3, 1, 2, 4],
]
XJoachim = pd.DataFrame(fact_table)
XJoachim.header = XJoachim.iloc[0:1] # sets the header to the first row 
XJoachim.columns = XJoachim.iloc[0] # Give the correct headers
XJoachim = XJoachim[1:] # Remove the header from row space
XJoachim[movies] # Perfect matrix presentation.

# print(
#   "$$"+
#   "UU_{pearson }(M_{joa})= "+
#   prettier_dataframe_to_latex(pd.DataFrame(similarity_matrix(XJoachim,pearson_similarity)))
#   +"$$"
#   )

# df = pd.DataFrame(
#   {
#     "people":knn_tuple(similarity_matrix(XJoachim,pearson_similarity), k=3)["3-idx"],
#     "similarity": knn_tuple(similarity_matrix(XJoachim,pearson_similarity), k=3)["3"],
#     "movie X": XJoachim[movies].iloc[knn_tuple(similarity_matrix(XJoachim,pearson_similarity), k=3)["3-idx"],0]
#   }
# )
# name = "joa"
# algo = "pearson"
# print(f"KNN_{ {'3,' +name} }(UU_{ {algo} }(M_{ {name} }))")
# print(prettier_dataframe_to_latex(df))
X_pers = 3
X_Movie = 3
FMat = XJoachim
idx = knn_tuple(similarity_matrix(FMat,eucledian_similarity), k=3)[f"{X_pers}-idx"]
pred_mat_avg = prediction_matrix(
  FMat,
  knn_tuple(similarity_matrix(FMat,eucledian_similarity), k=3),
 average_prediction
)
pred_wmat_avg = prediction_matrix(
  FMat,
  knn_tuple(similarity_matrix(FMat,eucledian_similarity), k=3),
 weighted_average_prediction
)
pred_wcmat_avg = prediction_matrix(
    FMat,
  knn_tuple(similarity_matrix(FMat,eucledian_similarity), k=3),
 weighted_average_corrected_prediction
)

pears_pred_mat_avg = prediction_matrix(
  FMat,
  knn_tuple(similarity_matrix(FMat,pearson_similarity), k=3),
 average_prediction
)
pears_pred_wmat_avg = prediction_matrix(
  FMat,
  knn_tuple(similarity_matrix(FMat,pearson_similarity), k=3),
 weighted_average_prediction
)
pears_pred_wcmat_avg = prediction_matrix(
    FMat,
  knn_tuple(similarity_matrix(FMat,pearson_similarity), k=3),
 weighted_average_corrected_prediction
)

pdf = pd.DataFrame(
  {"avg-euc": [pred_mat_avg[X_pers,X_Movie]], 
   "wavg-euc": [pred_wmat_avg[X_pers,X_Movie]], 
   "wcavg-euc": [pred_wcmat_avg[X_pers,X_Movie]],
   "avg-pearson": [pears_pred_mat_avg[X_pers,X_Movie]], 
   "wavg-pearson": [pears_pred_wmat_avg[X_pers,X_Movie]], 
   "wcavg-pearson": [pears_pred_wcmat_avg[X_pers,X_Movie]]
   }
).transpose()
print(prettier_dataframe_to_latex(pdf))


# pred_mat_avg[X_pers,X_Movie]

\begin{pmatrix}{lr}
 & 0 \\
avg-euc & 3.000000 \\
wavg-euc & 3.480107 \\
wcavg-euc & 4.188129 \\
avg-pearson & 3.000000 \\
wavg-pearson & -10.247924 \\
wcavg-pearson & -12.084611 \\
\end{pmatrix}



/var/folders/fp/6fd8hwdx18z6pbm4tp805qzh0000gn/T/ipykernel_81166/4191723395.py:13: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  XAnton.header = XAnton.iloc[0:1] # sets the header to the first row
/var/folders/fp/6fd8hwdx18z6pbm4tp805qzh0000gn/T/ipykernel_81166/4191723395.py:28: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  XBenjamin.header = XBenjamin.iloc[0:1] # sets the header to the first row
/var/folders/fp/6fd8hwdx18z6pbm4tp805qzh0000gn/T/ipykernel_81166/4191723395.py:43: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  XMads.header = XMads.iloc[0:1] # sets the header to the first row
/var/folders/fp/6fd8hwdx18z6pbm4tp805qzh0000gn/T